In [1]:
!pip install pot
!pip install torch
!pip install geomloss
!pip install wget
!pip install mdatagen


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

from geomloss import SamplesLoss

import ot

import os
import pickle as pkl
import copy

from sklearn.preprocessing import scale
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer

from utils import *
from SoftImpute import softimpute, cv_softimpute
#from DataSetLoader import dataset_loader, ground_truth
from SinkhornImputation import SinkhornImputation
from RR_imputer import RRimputer
import matplotlib.pyplot as plt
from Inject_Missing_Values import *
from Miss import *

import argparse
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test")
import pandas as pd

torch.set_default_tensor_type('torch.DoubleTensor')

c:\Users\zhossai3\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


Testing the new Missing function

In [3]:
d =  {
    "A": [10,10,20,40,70,40,50,80,10,40,20,60,50,30,20],
    "B": [1,1,2,4,7,4,5,8,1,4,2,6,5,3,2],
    "C":[15,15,30,60,105,60,75,120,15,60,30,90,75,45,30],
    "D":[45,30,60,32,58,63,25,55,46,78,92,60,34,27,35]

}
data = pd.DataFrame(d)

In [4]:
print(data)

     A  B    C   D
0   10  1   15  45
1   10  1   15  30
2   20  2   30  60
3   40  4   60  32
4   70  7  105  58
5   40  4   60  63
6   50  5   75  25
7   80  8  120  55
8   10  1   15  46
9   40  4   60  78
10  20  2   30  92
11  60  6   90  60
12  50  5   75  34
13  30  3   45  27
14  20  2   30  35


In [5]:
generator = Inject_Missing_Values()

MCAR

In [6]:
selected_columns = ['A','B']
miss_mcar,index_mcar = generator.MCAR(data,missing_rate=15)
print(miss_mcar.isna().sum())


print(miss_mcar)
print(miss_mcar.notna().sum())
print(index_mcar)

A    4
B    3
C    1
D    1
dtype: int64
       A    B      C     D
0    NaN  1.0   15.0  45.0
1    NaN  1.0   15.0  30.0
2   20.0  NaN    NaN  60.0
3   40.0  4.0   60.0  32.0
4   70.0  7.0  105.0  58.0
5    NaN  4.0   60.0  63.0
6   50.0  5.0   75.0   NaN
7   80.0  8.0  120.0  55.0
8   10.0  1.0   15.0  46.0
9   40.0  4.0   60.0  78.0
10  20.0  NaN   30.0  92.0
11  60.0  NaN   90.0  60.0
12  50.0  5.0   75.0  34.0
13  30.0  3.0   45.0  27.0
14   NaN  2.0   30.0  35.0
A    11
B    12
C    14
D    14
dtype: int64
0


In [7]:
missing_percentage = (miss_mcar.isnull().sum() / len(miss_mcar)) * 100
print(missing_percentage)


A    26.666667
B    20.000000
C     6.666667
D     6.666667
dtype: float64


In [8]:
total_missing_percentage = miss_mcar.isnull().sum().sum() / miss_mcar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")


Total Missing Percentage: 15.00%


MAR


In [9]:


dependencies_MAR = {
    "A": {
        "influencers": ["C","B"],
        "condition": lambda row: row["C"]+row["B"] > 30  # Just a boolean condition
          # Separate probability function
    },
    "B": {
        "influencers": ["D"],
        "condition": lambda row: True,
        "probability": lambda row: 0.2 if row["D"] < 30 else 0.1
    }
}

In [10]:
miss_mar,index_mar = generator.MAR(miss_mcar,dependencies_MAR,missing_rate=15)

In [11]:
print(index_mar)

0


In [12]:
print(miss_mar)

       A    B      C     D
0    NaN  1.0   15.0  45.0
1    NaN  NaN   15.0  30.0
2   20.0  NaN    NaN  60.0
3    NaN  4.0   60.0  32.0
4    NaN  7.0  105.0  58.0
5    NaN  4.0   60.0  63.0
6    NaN  5.0   75.0   NaN
7   80.0  8.0  120.0  55.0
8   10.0  1.0   15.0  46.0
9   40.0  4.0   60.0  78.0
10  20.0  NaN   30.0  92.0
11  60.0  NaN   90.0  60.0
12   NaN  5.0   75.0  34.0
13   NaN  3.0   45.0  27.0
14   NaN  2.0   30.0  35.0


In [13]:
total_missing_percentage = miss_mar.isnull().sum().sum() / miss_mar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")

Total Missing Percentage: 25.00%


MNAR

In [14]:
dependencies_MNAR = {
    "A": {
        "condition": lambda row: True,
        "probability": lambda row: 0.3 if row["A"] < 30 else 0.1  # High earners are more likely to not report salary
    },
    "C": {
        "condition": lambda row: row["C"] > 5, # High BP patients may avoid reporting BP readings
        "probability": lambda row: 0.2 
    },
    
}

In [15]:
miss_mnar,index_mnar = generator.MNAR(miss_mar,dependencies_MNAR,missing_rate=15)

In [16]:
print(miss_mnar)

       A    B      C     D
0    NaN  1.0   15.0  45.0
1    NaN  NaN   15.0  30.0
2    NaN  NaN    NaN  60.0
3    NaN  4.0   60.0  32.0
4    NaN  7.0  105.0  58.0
5    NaN  4.0    NaN  63.0
6    NaN  5.0    NaN   NaN
7   80.0  8.0  120.0  55.0
8   10.0  1.0    NaN  46.0
9   40.0  4.0   60.0  78.0
10  20.0  NaN   30.0  92.0
11  60.0  NaN    NaN  60.0
12   NaN  5.0   75.0  34.0
13   NaN  3.0    NaN  27.0
14   NaN  2.0   30.0  35.0


In [17]:
print(index_mnar)

0


In [18]:
total_missing_percentage = miss_mnar.isnull().sum().sum() / miss_mnar.size * 100
print(f"Total Missing Percentage: {total_missing_percentage:.2f}%")

Total Missing Percentage: 35.00%
